# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5b51564e20>
├── 'a' --> tensor([[-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107]])
└── 'x' --> <FastTreeValue 0x7f5b51564fa0>
    └── 'c' --> tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                        [ 0.0967,  0.3024,  0.9151, -0.4560],
                        [-1.2462,  1.1966, -1.0155, -0.9412]])

In [4]:
t.a

tensor([[-0.0465, -0.4072,  0.1912],
        [-0.0665, -0.4711, -1.1107]])

In [5]:
%timeit t.a

64 ns ± 0.0481 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5b51564e20>
├── 'a' --> tensor([[1.4404, 1.0194, 0.7217],
│                   [0.2605, 1.4276, 0.2351]])
└── 'x' --> <FastTreeValue 0x7f5b51564fa0>
    └── 'c' --> tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                        [ 0.0967,  0.3024,  0.9151, -0.4560],
                        [-1.2462,  1.1966, -1.0155, -0.9412]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0734 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107]]),
    x: Batch(
           c: tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                      [ 0.0967,  0.3024,  0.9151, -0.4560],
                      [-1.2462,  1.1966, -1.0155, -0.9412]]),
       ),
)

In [10]:
b.a

tensor([[-0.0465, -0.4072,  0.1912],
        [-0.0665, -0.4711, -1.1107]])

In [11]:
%timeit b.a

55.7 ns ± 0.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7037, -1.3687, -0.1832],
               [-1.3821, -0.7170,  0.8974]]),
    x: Batch(
           c: tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                      [ 0.0967,  0.3024,  0.9151, -0.4560],
                      [-1.2462,  1.1966, -1.0155, -0.9412]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

877 ns ± 0.249 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 19.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5a79635370>
├── 'a' --> tensor([[[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]],
│           
│                   [[-0.0465, -0.4072,  0.1912],
│                    [-0.0665, -0.4711, -1.1107]]])
└── 'x' --> <FastTreeValue 0x7f5a79650f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5b4c4e2fd0>
├── 'a' --> tensor([[-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107],
│                   [-0.0465, -0.4072,  0.1912],
│                   [-0.0665, -0.4711, -1.1107]])
└── 'x' --> <FastTreeValue 0x7f5b4c4e20a0>
    └── 'c' --> tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                        [ 0.0967,  0.3024,  0.9151, -0.4560],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 46.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]],
       
               [[-0.0465, -0.4072,  0.1912],
                [-0.0665, -0.4711, -1.1107]]]),
    x: Batch(
           c: tensor([[[ 0.0351,  0.3836, -0.9147,  0.0783],
                       [ 0.0967,  0.3024,  0.9151, -0.4560],
                       [-1.2462,  1.1966, -1.0155, -0.9412]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107],
               [-0.0465, -0.4072,  0.1912],
               [-0.0665, -0.4711, -1.1107]]),
    x: Batch(
           c: tensor([[ 0.0351,  0.3836, -0.9147,  0.0783],
                      [ 0.0967,  0.3024,  0.9151, -0.4560],
                      [-1.2462,  1.1966, -1.0155, -0.9412],
                      [ 0.0351,  0.3836, -0.9147,  0.0783],
                      [ 0.0967,  

In [28]:
%timeit Batch.cat(batches)

148 µs ± 490 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
